In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()


True

In [ ]:
from pyannote.audio import Pipeline
pipeline = Pipeline.from_pretrained(
  "pyannote/speaker-diarization-3.1",
  use_auth_token = os.getenv("HF_Token_Read"))

import torch
pipeline.to(torch.device("cuda"))

# run the pipeline on an audio file
diarization = pipeline("Telesales.wav", num_speakers=2)

# print the result
list_segments = []
for turn, _, speaker in diarization.itertracks(yield_label=True):
    list_segments.append((turn.start, turn.end, speaker))
    print(f"start={turn.start:.1f}s stop={turn.end:.1f}s speaker_{speaker}")


c:\Users\sirip\Anaconda3\envs\elasticsearch\Lib\site-packages\pyannote\audio\models\blocks\pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\ReduceOps.cpp:1823.)
  std = sequences.std(dim=-1, correction=1)


start=1.4s stop=4.7s speaker_SPEAKER_00
start=5.4s stop=6.1s speaker_SPEAKER_00
start=6.4s stop=8.2s speaker_SPEAKER_00
start=8.6s stop=12.6s speaker_SPEAKER_00
start=12.8s stop=19.1s speaker_SPEAKER_00
start=23.0s stop=23.3s speaker_SPEAKER_01
start=23.3s stop=23.7s speaker_SPEAKER_00
start=26.6s stop=27.5s speaker_SPEAKER_00
start=30.6s stop=33.3s speaker_SPEAKER_00
start=33.4s stop=37.8s speaker_SPEAKER_00
start=38.2s stop=38.9s speaker_SPEAKER_00
start=39.4s stop=43.6s speaker_SPEAKER_00
start=46.4s stop=56.2s speaker_SPEAKER_00
start=56.5s stop=60.0s speaker_SPEAKER_00
start=60.3s stop=61.2s speaker_SPEAKER_00
start=61.6s stop=68.8s speaker_SPEAKER_00
start=69.5s stop=73.1s speaker_SPEAKER_00
start=73.6s stop=76.0s speaker_SPEAKER_00
start=76.3s stop=77.3s speaker_SPEAKER_00
start=77.5s stop=82.8s speaker_SPEAKER_00
start=83.2s stop=88.1s speaker_SPEAKER_00
start=88.4s stop=95.6s speaker_SPEAKER_00
start=95.9s stop=99.6s speaker_SPEAKER_00
start=100.0s stop=103.0s speaker_SPEAKER_

In [25]:
list_segments

[(1.44846875, 4.654718750000001, 'SPEAKER_00'),
 (5.38034375, 6.072218750000001, 'SPEAKER_00'),
 (6.40971875, 8.24909375, 'SPEAKER_00'),
 (8.603468750000001, 12.56909375, 'SPEAKER_00'),
 (12.83909375, 19.11659375, 'SPEAKER_00'),
 (22.96409375, 23.26784375, 'SPEAKER_01'),
 (23.26784375, 23.74034375, 'SPEAKER_00'),
 (26.60909375, 27.53721875, 'SPEAKER_00'),
 (30.574718750000002, 33.274718750000005, 'SPEAKER_00'),
 (33.42659375, 37.76346875, 'SPEAKER_00'),
 (38.16846875, 38.92784375, 'SPEAKER_00'),
 (39.36659375, 43.55159375, 'SPEAKER_00'),
 (46.352843750000005, 56.174093750000004, 'SPEAKER_00'),
 (56.51159375, 59.970968750000004, 'SPEAKER_00'),
 (60.257843750000006, 61.236593750000004, 'SPEAKER_00'),
 (61.57409375, 68.77971875, 'SPEAKER_00'),
 (69.47159375000001, 73.11659375, 'SPEAKER_00'),
 (73.57221875, 75.95159375, 'SPEAKER_00'),
 (76.25534375000001, 77.30159375000001, 'SPEAKER_00'),
 (77.53784375000001, 82.80284375000001, 'SPEAKER_00'),
 (83.17409375000001, 88.05096875000001, 'SPEAKE

In [35]:
from pydub import AudioSegment
import os

def preprocess_and_segment_audio(audio_path, segments, output_folder="segmented_audio", merge_threshold=0.7, padding=0.5, save_in_one_folder=True):
    """
    Preprocesses speaker diarization segments by merging close segments and splits audio accordingly with optional padding.

    Parameters:
        - audio_path (str): Path to the audio file.
        - segments (list of tuples): List of (start, end, speaker) segments.
        - output_folder (str): Folder to save segmented audio files.
        - merge_threshold (float): Maximum gap (in seconds) between segments to merge.
        - padding (float): Additional time (in seconds) before and after each segment.

    Returns:
        - None (saves segmented audio files)
    """
    
    # input
    audio = AudioSegment.from_wav(audio_path)
    
    # make output folder
    os.makedirs(output_folder, exist_ok=True)
    

    merged_segments = []
    for segment in segments:
        start, end, speaker = segment
        if merged_segments and speaker == merged_segments[-1][2]:  
            prev_start, prev_end, prev_speaker = merged_segments[-1]
            if start - prev_end <= merge_threshold:  
                merged_segments[-1] = (prev_start, end, prev_speaker)  
            else:
                merged_segments.append(segment)
        else:
            merged_segments.append(segment)
    

    for i, (start, end, speaker) in enumerate(merged_segments):
        # Add padding to the start and end time
        start = max(0, start - padding)  
        end = min(len(audio) / 1000, end + padding)

        # min to milliseconds
        start_ms = int(start * 1000)
        end_ms = int(end * 1000)

        # cut audio
        segment = audio[start_ms:end_ms]

        
        # speaker_folder = os.path.join(output_folder, speaker)
        # os.makedirs(speaker_folder, exist_ok=True)

        if save_in_one_folder:
            speaker_folder = output_folder  
        else:
            # name
            speaker_folder = os.path.join(output_folder, speaker)  # แยกตาม speaker
            os.makedirs(speaker_folder, exist_ok=True)

        # save
        segment_path = os.path.join(speaker_folder, f"{speaker}_{i+1:03d}_{start:.2f}s-{end:.2f}s.wav")
        segment.export(segment_path, format="wav")

        print(f"Saved: {segment_path}")

    print("Finished segmenting audio.")

preprocess_and_segment_audio("Telesales.wav", list_segments, output_folder="segmented_audio", merge_threshold=0.7, padding=0.5)


Saved: segmented_audio\SPEAKER_00_001_0.95s-5.15s.wav
Saved: segmented_audio\SPEAKER_00_002_4.88s-19.62s.wav
Saved: segmented_audio\SPEAKER_01_003_22.46s-23.77s.wav
Saved: segmented_audio\SPEAKER_00_004_22.77s-24.24s.wav
Saved: segmented_audio\SPEAKER_00_005_26.11s-28.04s.wav
Saved: segmented_audio\SPEAKER_00_006_30.07s-44.05s.wav
Saved: segmented_audio\SPEAKER_00_007_45.85s-110.94s.wav
Saved: segmented_audio\SPEAKER_00_008_110.82s-114.88s.wav
Saved: segmented_audio\SPEAKER_00_009_114.65s-129.20s.wav
Saved: segmented_audio\SPEAKER_00_010_129.06s-134.40s.wav
Saved: segmented_audio\SPEAKER_00_011_134.38s-145.61s.wav
Saved: segmented_audio\SPEAKER_00_012_145.85s-147.83s.wav
Saved: segmented_audio\SPEAKER_00_013_148.01s-152.81s.wav
Saved: segmented_audio\SPEAKER_00_014_152.87s-163.56s.wav
Saved: segmented_audio\SPEAKER_00_015_163.69s-177.30s.wav
Saved: segmented_audio\SPEAKER_00_016_177.11s-186.21s.wav
Saved: segmented_audio\SPEAKER_00_017_186.10s-191.67s.wav
Saved: segmented_audio\SPEAKER

In [27]:
merge_threshold = 0.7

merged_segments = []
for segment in list_segments:
    start, end, speaker = segment
    if merged_segments and speaker == merged_segments[-1][2]:  # ตรวจสอบ speaker เดียวกัน
        prev_start, prev_end, prev_speaker = merged_segments[-1]
        if start - prev_end <= merge_threshold:  # เช็คว่าช่องว่างระหว่าง segment น้อยกว่า 0.7 วินาทีหรือไม่
            merged_segments[-1] = (prev_start, end, prev_speaker)  # รวม segment
        else:
            merged_segments.append(segment)
    else:
        merged_segments.append(segment)

In [28]:
merged_segments

[(1.44846875, 4.654718750000001, 'SPEAKER_00'),
 (5.38034375, 19.11659375, 'SPEAKER_00'),
 (22.96409375, 23.26784375, 'SPEAKER_01'),
 (23.26784375, 23.74034375, 'SPEAKER_00'),
 (26.60909375, 27.53721875, 'SPEAKER_00'),
 (30.574718750000002, 43.55159375, 'SPEAKER_00'),
 (46.352843750000005, 110.44409375000001, 'SPEAKER_00'),
 (111.32159375, 114.37596875000001, 'SPEAKER_00'),
 (115.15221875, 128.70284375, 'SPEAKER_00'),
 (129.56346875, 133.90034375000002, 'SPEAKER_00'),
 (134.87909375, 145.10534375, 'SPEAKER_00'),
 (146.35409375, 147.33284375000002, 'SPEAKER_00'),
 (148.51409375, 152.31096875, 'SPEAKER_00'),
 (153.37409375000001, 163.06034375000002, 'SPEAKER_00'),
 (164.19096875000002, 176.79659375, 'SPEAKER_00'),
 (177.60659375, 185.70659375000002, 'SPEAKER_00'),
 (186.60096875000002, 191.17409375000003, 'SPEAKER_00'),
 (192.22034375, 196.13534375, 'SPEAKER_00'),
 (196.91159375, 218.64659375000002, 'SPEAKER_00'),
 (222.08909375000002, 226.17284375000003, 'SPEAKER_01'),
 (231.64034375000